In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)

# CONSTANTS
TARGET = "cases_new_increase_tmr"
START_DATE = pd.to_datetime("2021-07-01")
END_DATE = START_DATE + pd.DateOffset(months=6)
NO_DAYS = (END_DATE - START_DATE).days + 1
dates = pd.date_range(start=START_DATE, end=END_DATE, freq="D")
print(f"No. days: {NO_DAYS}")
print(f"Date range: {START_DATE} to {END_DATE}")

# Base GitHub URL
BASE_URL = "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/"

# Relevant CSVs
files = {
    "cases_malaysia": BASE_URL + "epidemic/cases_malaysia.csv",
    "tests_malaysia": BASE_URL + "epidemic/tests_malaysia.csv",
    "checkin_malaysia": BASE_URL + "mysejahtera/checkin_malaysia.csv",
    # "deaths_malaysia": BASE_URL + "epidemic/deaths_malaysia.csv",
    # "hospital": BASE_URL + "epidemic/hospital.csv",
    # "icu": BASE_URL + "epidemic/icu.csv",
    # "vax_malaysia": BASE_URL + "vaccination/vax_malaysia.csv",
    # "trace_malaysia": BASE_URL + "mysejahtera/trace_malaysia.csv",
}

No. days: 185
Date range: 2021-07-01 00:00:00 to 2022-01-01 00:00:00


In [100]:
# Extract and laod all CSVs into DataFrames
dfs = {name: pd.read_csv(url) for name, url in files.items()}

# Clean data
for k, df in dfs.items():
    # Set date column as data
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
        df = df[(df["date"] >= START_DATE) & (df["date"] <= END_DATE)]

    # Drop columns where ALL values are null
    df = df.dropna(axis=1, how="all")

    # Clean data
    if k == "trace_malaysia":
        # Handle duplicates
        df = df.groupby("date", as_index=False).mean()

        # Interpolate data for missing dates
        df["date"] = pd.to_datetime(df["date"])
        df.set_index("date", inplace=True)
        df.sort_index(inplace=True)
        df = df.reindex(dates)
        cols = ["casual_contacts", "hide_large", "hide_small"]
        df[cols] = df[cols].interpolate(method="linear")
        df = df.reset_index().rename(columns={"index": "date"})

        # Fix columns type
        df[cols] = df[cols].round().astype(int)

    # Remove columns with one value only
    df = df.loc[:, df.nunique(dropna=True) > 1]

    # Save back into dictionary
    dfs[k] = df

    # Check shape
    output_filename = f"{k}.csv"
    print(f"{k}: {df.shape}")

# Merge data
data = pd.DataFrame({'date': dates})
for k, df in dfs.items():
    if k == "population":
        continue
    if df.shape[0] != NO_DAYS:
        df = df.drop(columns=["state"])
        df = df.groupby("date").sum()
    data = pd.merge(data, df, on="date", how="left")

cases_malaysia: (185, 24)
tests_malaysia: (185, 3)
checkin_malaysia: (185, 4)


In [101]:
# Include target cases_new_increase_tmr
data["cases_new_increase_tmr"] = (
    data["cases_new"].shift(-1) > data["cases_new"]
).astype(int)

# Drop last row (no target)
data = data[:-1]

# Features selection
base_features = [
    "cases_new",
    "cases_active",
    "cases_cluster",
    "tests_total",
    "mobility_density",
]

feature_cols = base_features.copy()

# Combine features
data["tests_total"] = data["rtk-ag"] + data["pcr"]  # combine tests
data["mobility_density"] = (
    data["checkins"] / data["unique_loc"]
)  # density of people per location
data["mobility_density"] = (
    data["mobility_density"].replace([float("inf"), -float("inf")], 0).fillna(0)
)

lag_days = [1]
for col in base_features:
    # Lag features
    for day in lag_days:
        feat_name =f"{col}_shift{day}" 
        data[feat_name] = data[col].shift(day)  # previous day
        feature_cols.append(feat_name)

    # Rolling averages (7-day)
    feat_name = f"{col}_7d_avg"
    data[feat_name] = data[col].rolling(window=7).mean()  # 7-day avg
    feature_cols.append(feat_name)

    # Percent change
    feat_name = f"{col}_pct_change"
    data[feat_name] = data[col].pct_change()  # daily pct change
    feature_cols.append(feat_name)

# Day of week
data["day_of_week"] = data["date"].dt.dayofweek  # 0=Monday, 6=Sunday
feature_cols.append("day_of_week")

# Drop rows with NaN caused by lag/rolling
data = data.dropna().reset_index(drop=True)  # clean data

# Target and features
y = data["cases_new_increase_tmr"]  # target
X = data[feature_cols]  # features

# Output shapes
print("X shape:", X.shape)
print("y shape:", y.shape)
print("y distribution: ", y.value_counts())
print("Selected features:", feature_cols)

X shape: (178, 21)
y shape: (178,)
y distribution:  cases_new_increase_tmr
0    93
1    85
Name: count, dtype: int64
Selected features: ['cases_new', 'cases_active', 'cases_cluster', 'tests_total', 'mobility_density', 'cases_new_shift1', 'cases_new_7d_avg', 'cases_new_pct_change', 'cases_active_shift1', 'cases_active_7d_avg', 'cases_active_pct_change', 'cases_cluster_shift1', 'cases_cluster_7d_avg', 'cases_cluster_pct_change', 'tests_total_shift1', 'tests_total_7d_avg', 'tests_total_pct_change', 'mobility_density_shift1', 'mobility_density_7d_avg', 'mobility_density_pct_change', 'day_of_week']


In [102]:
def train_evaluate_model_split(
    model, param_grid, X, y, scale_data=False, n_splits=5
):
    """
    Train and evaluate a model using TimeSeriesSplit + GridSearchCV.
    Splits data into train/validation folds sequentially, and uses the
    last fold as the final test set.
    Returns results DataFrame with metrics, best params, timings, and classification report.
    """

    model_name = type(model).__name__
    print(f"\n=== Training {model_name} with TimeSeriesSplit ===")

    # Initialize time series split
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Use the last split as train/validation/test separation
    splits = list(tscv.split(X))
    train_idx, test_idx = splits[-1]  # last split
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Scaling if needed
    if scale_data:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    else:
        X_train = X_train.values
        X_test = X_test.values

    # GridSearchCV (with TimeSeriesSplit on training set only)
    start_train = time.time()
    grid = GridSearchCV(model, param_grid, cv=TimeSeriesSplit(n_splits=n_splits-1), scoring="f1", n_jobs=-1)
    grid.fit(X_train, y_train)
    train_time = time.time() - start_train

    print(f"[{model_name}] Best params: {grid.best_params_}")
    print(f"[{model_name}] Training time: {train_time:.2f} seconds")

    best_model = grid.best_estimator_

    # Inference
    start_infer = time.time()
    y_pred = best_model.predict(X_test)
    infer_time = time.time() - start_infer
    print(f"[{model_name}] Inference time: {infer_time:.4f} seconds")

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    print(
        f"[{model_name}] Results - Accuracy: {acc:.4f}, "
        f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}"
    )
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Save to results DataFrame
    df_results = pd.DataFrame(
        [
            {
                "model": model_name,
                "accuracy": acc,
                "precision": precision,
                "recall": recall,
                "f1_score": f1,
                "best_params": grid.best_params_,
                "train_time_sec": train_time,
                "infer_time_sec": infer_time,
                "confusion_matrix": cm,
                "classification_report": report,
            }
        ]
    )

    df_avg = pd.DataFrame(
        [
            {
                "model": model_name,
                "average_accuracy": acc,
                "average_precision": precision,
                "average_recall": recall,
                "average_f1": f1,
                "avg_train_time_sec": train_time,
                "avg_infer_time_sec": infer_time,
            }
        ]
    )

    print(f"\n=== Finished {model_name} ===")
    print(
        f"Accuracy: {acc:.4f}, Precision: {precision:.4f}, "
        f"Recall: {recall:.4f}, F1: {f1:.4f}, "
        f"Train Time: {train_time:.2f}s, Infer Time: {infer_time:.4f}s"
    )

    return df_results, df_avg

In [103]:
# Define models
models = {
    "LogisticRegression": LogisticRegression(class_weight="balanced", max_iter=2000),
    "RandomForest": RandomForestClassifier(class_weight="balanced", n_jobs=-1),
    "XGB": xgb.XGBClassifier(
        eval_metric="aucpr",
        scale_pos_weight=len(y[y == 0]) / len(y[y == 1]),
        n_jobs=-1,
        verbosity=0,
    ),
}

# Define smaller param grids
param_grids = {
    "LogisticRegression": [
        {  # L2 penalty (most stable for small datasets)
            "penalty": ["l2"],
            "solver": ["saga"],
            "C": np.logspace(-2, 1, 5),
        },
        {  # ElasticNet (optional)
            "penalty": ["elasticnet"],
            "solver": ["saga"],
            "C": np.logspace(-2, 1, 5),
            "l1_ratio": [0.2, 0.5, 0.8],
        },
    ],
    "RandomForest": {
        "n_estimators": [100, 200],
        "max_depth": [5, 10, None],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 2],
        "max_features": ["sqrt"],
    },
    "XGB": {
        "n_estimators": [100, 200],
        "max_depth": [3, 5],
        "learning_rate": [0.05, 0.1],
        "subsample": [0.8, 1.0],
        "colsample_bytree": [0.8, 1.0],
        "gamma": [0, 1],
        "reg_alpha": [0, 1],
        "reg_lambda": [1, 5],
    },
}

all_results = []
all_avg = []
for name, model in models.items():
    scale_needed = name in ["LogisticRegression", "KNN", "SVC"]
    print(f"\n--- Running {name} ---")
    df_result, df_avg = train_evaluate_model_split(
        model, param_grids[name], X, y, scale_data=scale_needed
    )

    # Tag model name explicitly
    df_result["model"] = name
    df_avg["model"] = name
    all_results.append(df_result)
    all_avg.append(df_avg)

# Combine all results
df_results_combined = pd.concat(all_results, ignore_index=True)
df_avg_combined = pd.concat(all_avg, ignore_index=True)

# Display
print("\nResults per model:")
print(
    df_results_combined[
        ["model", "accuracy", "f1_score", "best_params", "confusion_matrix"]
    ]
)
print("\nAverage metrics per model:")
print(df_avg_combined)


--- Running LogisticRegression ---

=== Training LogisticRegression with TimeSeriesSplit ===


[LogisticRegression] Best params: {'C': np.float64(0.31622776601683794), 'penalty': 'l2', 'solver': 'saga'}
[LogisticRegression] Training time: 0.39 seconds
[LogisticRegression] Inference time: 0.0005 seconds
[LogisticRegression] Results - Accuracy: 0.8966, Precision: 0.8571, Recall: 0.9231, F1: 0.8889

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.88      0.90        16
           1       0.86      0.92      0.89        13

    accuracy                           0.90        29
   macro avg       0.90      0.90      0.90        29
weighted avg       0.90      0.90      0.90        29


=== Finished LogisticRegression ===
Accuracy: 0.8966, Precision: 0.8571, Recall: 0.9231, F1: 0.8889, Train Time: 0.39s, Infer Time: 0.0005s

--- Running RandomForest ---

=== Training RandomForestClassifier with TimeSeriesSplit ===
[RandomForestClassifier] Best params: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_sa